In [ ]:
import os
import pandas as pd
from aicsimageio import transforms, AICSImage
import skimage
import skimage.morphology 
import scipy
import numpy
import json

def add_granularity(csvname,cell_stage=None,cols2D=['max_projection_x','max_projection_y','max_projection_z','mean_projection_x','mean_projection_y','mean_projection_z','center_slice',]):

    def measure_granularity(full_image,ProjType,feature_dict):
        # Channels 1,2,3 are dna, mem, structure intensities
        # Channels 4,5,6 are dna, mem, structure segmentations
        channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
        compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
        for eachchan in channels.keys():
            image = full_image[0,eachchan,0,:,:]
            image = numpy.where(image<0,False,image)
            image = skimage.exposure.rescale_intensity(image, in_range='image', out_range=(0,1))
            for eachcompartment in compartments.keys():
                labels = full_image[0,eachcompartment,0,:,:]
                binary_labels = labels>0
                image = image*binary_labels
                startmean = numpy.mean(image)           

                #Calculate granularity
                pixels=image.copy()
                subfactor=.5
                samp_size=.25
                radius=5
                eros=15

                orig = image.copy()

                # Rescale image 
                new_shape = numpy.array(pixels.shape)
                new_shape = new_shape * subfactor
                i, j = (
                    numpy.mgrid[0 : new_shape[0], 0 : new_shape[1]].astype(float)
                    / subfactor
                )
                pixels = scipy.ndimage.map_coordinates(pixels, (i, j), order=1)

                mask = numpy.ones_like(pixels)
                prepixels = pixels.copy()

                # Do initial processing based on radius
                back_shape = new_shape * samp_size
                i, j = (
                    numpy.mgrid[0 : back_shape[0], 0 : back_shape[1]].astype(float)
                    / samp_size
                )
                back_pixels = scipy.ndimage.map_coordinates(pixels, (i, j), order=1)
                back_mask = (
                    scipy.ndimage.map_coordinates(mask.astype(float), (i, j)) > 0.9
                )

                # "Radius of structuring element"
                selem = skimage.morphology.disk(radius, dtype=bool)
                back_pixels_mask = numpy.zeros_like(back_pixels)
                back_pixels_mask[back_mask == True] = back_pixels[back_mask == True]
                back_pixels = skimage.morphology.erosion(back_pixels_mask, footprint=selem)
                back_pixels_mask = numpy.zeros_like(back_pixels)
                back_pixels_mask[back_mask == True] = back_pixels[back_mask == True]
                back_pixels = skimage.morphology.dilation(back_pixels_mask, footprint=selem)
                i, j = numpy.mgrid[0 : new_shape[0], 0 : new_shape[1]].astype(float)
                #
                # Make sure the mapping only references the index range of
                # back_pixels.
                #
                i *= float(back_shape[0] - 1) / float(new_shape[0] - 1)
                j *= float(back_shape[1] - 1) / float(new_shape[1] - 1)
                back_pixels = scipy.ndimage.map_coordinates(
                    back_pixels, (i, j), order=1
                )
                pixels -= back_pixels
                pixels[pixels < 0] = 0

                ero = pixels.copy()

                footprint = skimage.morphology.disk(1, dtype=bool)

                for eachcount in range(0,eros):
                    prev_ero = ero
                    eachcount +=1
                    ero_mask = numpy.zeros_like(ero)
                    ero_mask[mask == True] = ero[mask == True]
                    ero = skimage.morphology.erosion(ero_mask, footprint=footprint)

                    prevmean = numpy.mean(prev_ero)
                    currentmean = numpy.mean(ero)

                    gs = (prevmean - currentmean) * 100 / startmean
                    feature_name = f"Granularity_{eachcount}_{compartments[eachcompartment]}_{channels[eachchan]}_{ProjType}"

                    feature_dict[feature_name].append(gs)
                
        return feature_dict

    #cols2D = ['max_projection_x','max_projection_y','max_projection_z','mean_projection_x','mean_projection_y','mean_projection_z','center_slice',]
    feature_dict={}
    channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
    compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
    for eachcount in range(1,16):
        for eachchan in channels.keys():
            for ProjType in cols2D:
                for eachcompartment in compartments.keys():
                    feature_dict[f"Granularity_{eachcount}_{compartments[eachcompartment]}_{channels[eachchan]}_{ProjType}"] = []
    
    count = 0
    df = pd.read_csv(csvname)
    if cell_stage!=None:
        csvname = csvname[:-4]+cell_stage+'.csv'
        df = df.query(f"cell_stage == '{cell_stage}'")
    allcols2D='-'.join(cols2D)
    csvname = csvname[:-4]+allcols2D+'.csv'
    for _, eachrow in df.iterrows():
        for ProjType in cols2D:
            im = AICSImage(eachrow[ProjType]).data
            feature_dict = measure_granularity(im,ProjType,feature_dict)
            count +=1 
            if count % 100 == 0:
                print(count)
                with open(f"granularity_{cell_stage}_{allcols2D}.json","w") as outfile:
                    json.dump(feature_dict, outfile)
    
    df = pd.concat([df,pd.DataFrame(feature_dict)],axis=1)
    df.to_csv(csvname,index=False)
    
if __name__=='__main__':
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("csv")
    parser.add_argument("cell_stage")
    parser.add_argument("proj2D",
                       type=lambda s: [item for item in s.split(",")])
    args = parser.parse_args()
    add_granularity(args.csv,cell_stage=args.cell_stage,cols2D=args.proj2D)

In [ ]:
import pandas
import skimage.exposure
import numpy
import mahotas
from aicsimageio import AICSImage
import json

def add_2D_textures(csvname,cell_stage=None,cols2D=['max_projection_x','max_projection_y','max_projection_z','mean_projection_x','mean_projection_y','mean_projection_z','center_slice',]):

    def measure_2D_textures(full_image,ProjType,sizes,feature_dict):
        # Channels 1,2,3 are dna, mem, structure intensities
        # Channels 4,5,6 are dna, mem, structure segmentations
        channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
        compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
        features = {}
        direction_feature_names = """AngularSecondMoment Contrast Correlation Variance
    InverseDifferenceMoment SumAverage SumVariance SumEntropy Entropy
    DifferenceVariance DifferenceEntropy InfoMeas1 InfoMeas2""".split()
        n_directions = 4
        for eachchan in channels.keys():
            image = full_image[0,eachchan,0,:,:]
            pixel_data = skimage.exposure.rescale_intensity(image,out_range='uint8')
            for eachcomp in compartments.keys():
                labels = full_image[0,eachcomp,0,:,:]
                binary_labels = labels>0
                label_data = pixel_data*binary_labels
                for size in sizes:
                    texture_features = mahotas.features.haralick(
                            label_data, distance=size
                        )
                        
                    for direction, direction_features in enumerate(texture_features):
                        for feature_name, feature in zip(direction_feature_names, direction_features):
                            object_name = "Texture_2D_{:}_{:}_{:}_{:}_{:02d}_{:02d}".format(compartments[eachcomp],channels[eachchan],ProjType, feature_name, direction, size)
                            feature_dict[object_name].append(feature)
        return feature_dict

    sizes = [10]
    #cols2D = ['max_projection_x','max_projection_y','max_projection_z','mean_projection_x','mean_projection_y','mean_projection_z','center_slice',]
    #cols2D = ['max_projection_x']
    feature_dict={}
    direction_feature_names = """AngularSecondMoment Contrast Correlation Variance
    InverseDifferenceMoment SumAverage SumVariance SumEntropy Entropy
    DifferenceVariance DifferenceEntropy InfoMeas1 InfoMeas2""".split()
    directions=13
    channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
    compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
    for eachchan in channels.keys():
        for eachcomp in compartments.keys():
            for direction in range(4):
                for feature_name in direction_feature_names:
                    for size in sizes:
                        for ProjType in cols2D:
                            object_name = "Texture_2D_{:}_{:}_{:}_{:}_{:02d}_{:02d}".format(compartments[eachcomp],channels[eachchan],ProjType, feature_name, direction, size)
                            feature_dict[object_name]=[]
    
    df = pandas.read_csv(csvname)
    if cell_stage!=None:
        csvname = csvname[:-4]+cell_stage+'.csv'
        df = df.query(f"cell_stage == '{cell_stage}'")
    allcols2D='-'.join(cols2D)
    csvname = csvname[:-4]+allcols2D+'.csv'
    count = 0
    for _, eachrow in df.iterrows():
        for ProjType in cols2D:
            im = AICSImage(eachrow[ProjType]).data
            feature_dict = measure_2D_textures(im,ProjType,sizes,feature_dict)
            count +=1 
            if count % 100 == 0:
                print(count)
                with open(f"texture_5_{cell_stage}_{allcols2D}.json","w") as outfile:
                    json.dump(feature_dict, outfile)
    df = pandas.concat([df,pandas.DataFrame(feature_dict)],axis=1)
    df.to_csv(csvname,index=False)
    
if __name__=='__main__':
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("csv")
    parser.add_argument("cell_stage")
    parser.add_argument("proj2D",
                       type=lambda s: [item for item in s.split(",")])
    args = parser.parse_args()
    add_2D_textures(args.csv,cell_stage=args.cell_stage,cols2D=args.proj2D)

In [ ]:
import pandas
import skimage.exposure
import numpy
from aicsimageio import AICSImage
import json

def add_2D_shapes(csvname,cell_stage=None,cols2D=['max_projection_x','max_projection_y','max_projection_z','mean_projection_x','mean_projection_y','mean_projection_z','center_slice',]):

    def measure_2D_shapes(full_image,ProjType,feature_dict):
        # Channels 1,2,3 are dna, mem, structure intensities
        # Channels 4,5,6 are dna, mem, structure segmentations
        #channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
        compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
        features = {}
        desired_properties = ["label","area","perimeter","bbox","bbox_area","major_axis_length",
                "minor_axis_length","orientation","centroid","equivalent_diameter","extent",
                "eccentricity","convex_area","solidity","euler_number","moments","moments_central",
                "moments_normalized"]
        for eachcomp in compartments.keys():
            labels = skimage.measure.label(full_image[0,eachcomp,0,:,:])
            starting_props = ["area","perimeter","major_axis_length","minor_axis_length","eccentricity","orientation","centroid-1","centroid-0",
             "bbox_area","bbox-1","bbox-3","bbox-0","bbox-2","extent","solidity","euler_number","convex_area",
             "equivalent_diameter"]
            for x in range(4):
                for y in range(4):
                    starting_props.append(f"moments_normalized-{x}-{y}")
                    if x!= 3:
                        starting_props.append(f"moments_central-{x}-{y}")
            props=skimage.measure.regionprops_table(labels,properties=desired_properties)
            for feature_name in starting_props:
                object_name = "Shape_2D_{:}_{:}_{:}".format(compartments[eachcomp],ProjType, feature_name)
                if len(props[feature_name])==0:
                    feature_dict[object_name].append(numpy.nan)
                elif len(props[feature_name])==1:
                    feature_dict[object_name].append(float(props[feature_name]))
                else:
                    feature_dict[object_name].append(float(numpy.mean(props[feature_name])))
        return feature_dict

    #cols2D = ['max_projection_x','max_projection_y','max_projection_z','mean_projection_x','mean_projection_y','mean_projection_z','center_slice',]
    #cols2D = ['max_projection_x']
    feature_dict={}
    starting_props = ["area","perimeter","major_axis_length","minor_axis_length","eccentricity","orientation","centroid-1","centroid-0",
     "bbox_area","bbox-1","bbox-3","bbox-0","bbox-2","extent","solidity","euler_number","convex_area",
     "equivalent_diameter"]
    for x in range(4):
        for y in range(4):
            starting_props.append(f"moments_normalized-{x}-{y}")
            if x!= 3:
                starting_props.append(f"moments_central-{x}-{y}")
                starting_props.append(f"moments_central-{x}-{y}")
    compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
    for eachcomp in compartments.keys():
        for feature_name in starting_props:
            for ProjType in cols2D:
                object_name = "Shape_2D_{:}_{:}_{:}".format(compartments[eachcomp],ProjType, feature_name)
                feature_dict[object_name]=[]
    
    df = pandas.read_csv(csvname)
    if cell_stage!=None:
        csvname = csvname[:-4]+cell_stage+'.csv'
        df = df.query(f"cell_stage == '{cell_stage}'")
    allcols2D='-'.join(cols2D)
    csvname = csvname[:-4]+allcols2D+'.csv'
    count = 0 
    for _, eachrow in df.iterrows():
        for ProjType in cols2D:
            im = AICSImage(eachrow[ProjType]).data
            feature_dict = measure_2D_shapes(im,ProjType,feature_dict)
            count +=1 
            if count % 100 == 0:
                print(count)
                with open(f"shape_{cell_stage}_{allcols2D}.json","w") as outfile:
                    json.dump(feature_dict, outfile)
    df = pandas.concat([df,pandas.DataFrame(feature_dict)],axis=1)
    df.to_csv(csvname,index=False)
    
if __name__=='__main__':
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("csv")
    parser.add_argument("cell_stage")
    parser.add_argument("proj2D",
                       type=lambda s: [item for item in s.split(",")])
    args = parser.parse_args()
    add_2D_shapes(args.csv,cell_stage=args.cell_stage,cols2D=args.proj2D)


In [ ]:
import pandas
import skimage.exposure
import numpy
from aicsimageio import AICSImage
import json

def add_3D_shapes_intensities(csvname,cell_stage=None):

    def measure_3D_shapes(full_image,feature_dict):
        # Channels 1,2,3 are dna, mem, structure intensities
        # Channels 4,5,6 are dna, mem, structure segmentations
        #channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
        compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
        channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
        features = {}
        desired_properties = [
            "area",
            "centroid",
            "major_axis_length",
            "minor_axis_length",
            "extent",
            "equivalent_diameter",
        ]

        intensity_properties = [
            'intensity_mean',
            'intensity_max',
            'intensity_min',
        ]
        for eachcomp in compartments.keys():
            labels = skimage.measure.label(full_image[0,eachcomp,:,:,:])
            try:
                props=skimage.measure.regionprops_table(labels,properties=desired_properties)
                for feature_name in desired_properties:
                    object_name = "Shape_3D_{:}_{:}".format(compartments[eachcomp], feature_name)
                    try:
                        if len(props[feature_name])==0:
                            feature_dict[object_name].append(numpy.nan)
                        elif len(props[feature_name])==1:
                            feature_dict[object_name].append(float(props[feature_name]))
                        else:
                            feature_dict[object_name].append(float(numpy.mean(props[feature_name])))
                    except KeyError:
                        feature_dict[object_name].append(numpy.nan)
            except ValueError:
                for feature_name in desired_properties:
                    object_name = "Shape_3D_{:}_{:}".format(compartments[eachcomp], feature_name)
                    feature_dict[object_name].append(numpy.nan)
            for eachchan in channels.keys():
                image = full_image[0,eachchan,:,:,:]
                binary_labels = labels>0
                label_data = image*binary_labels
                props=skimage.measure.regionprops_table(labels,label_data,properties=intensity_properties)
                for feature_name in intensity_properties:
                    object_name = "Intensity_3D_{:}_{:}_{:}".format(compartments[eachcomp], channels[eachchannel],feature_name)
                    if len(props[feature_name])==0:
                        feature_dict[object_name].append(numpy.nan)
                    elif len(props[feature_name])==1:
                        feature_dict[object_name].append(float(props[feature_name]))
                    else:
                        feature_dict[object_name].append(float(numpy.mean(props[feature_name])))
        return feature_dict

    #cols2D = ['max_projection_x','max_projection_y','max_projection_z','mean_projection_x','mean_projection_y','mean_projection_z','center_slice',]
    #cols2D = ['max_projection_x']
    feature_dict={}
    desired_properties = [
        "area",
        "centroid",
        "major_axis_length",
        "minor_axis_length",
        "extent",
        "equivalent_diameter",
    ]
    
    intensity_properties = [
        'intensity_mean',
        'intensity_max',
        'intensity_min',
    ]
    channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
    compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
    for eachcomp in compartments.keys():
        for feature_name in desired_properties:
            object_name = "Shape_3D_{:}_{:}".format(compartments[eachcomp], feature_name)
            feature_dict[object_name]=[]
        for eachchannel in channels:
            for feature_name in intensity_properties:
                object_name = "Intensity_3D_{:}_{:}_{:}".format(compartments[eachcomp], channels[eachchannel],feature_name)
                feature_dict[object_name]=[]
    
    df = pandas.read_csv(csvname)
    if cell_stage!=None:
        csvname = csvname[:-4]+cell_stage+'.csv'
        df = df.query(f"cell_stage == '{cell_stage}'")
    csvname = csvname[:-4]+'.csv'
    count = 0 
    for _, eachrow in df.iterrows():
        im = AICSImage(eachrow["3d_image"].replace("register", "merge").replace("zarr", "tiff")).data
        feature_dict = measure_3D_shapes(im,feature_dict)
        count +=1 
        if count % 100 == 0:
            print(count)
            with open(f"intensity_shape_{cell_stage}_3D.json","w") as outfile:
                json.dump(feature_dict, outfile)
    df = pandas.concat([df,pandas.DataFrame(feature_dict)],axis=1)
    df.to_csv(csvname,index=False)
    
if __name__=='__main__':
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("csv")
    parser.add_argument("cell_stage")
    args = parser.parse_args()
    add_3D_shapes_intensities(args.csv,cell_stage=args.cell_stage)
